# REINFORCE Algorithm: Implementation and Analysis

## Overview
REINFORCE is the foundational policy gradient algorithm that directly uses Monte Carlo returns to estimate policy gradients.

### Learning Objectives
1. Understand REINFORCE algorithm
2. Implement REINFORCE from scratch
3. Train on CartPole environment
4. Analyze learning curves and performance

## 1. REINFORCE Algorithm

### Algorithm Description

REINFORCE uses the policy gradient theorem with Monte Carlo returns:

$$\nabla_\theta J(\theta) = \mathbb{E}[\nabla_\theta \log \pi_\theta(a|s) G_t]$$

where $G_t = \sum_{k=0}^{T-t-1} \gamma^k r_{t+k}$ is the discounted return.

### Update Rule

$$\theta \leftarrow \theta + \alpha \nabla_\theta \log \pi_\theta(a|s) G_t$$

### Pseudocode

```
Initialize policy π_θ
for episode = 1 to num_episodes:
    Collect trajectory: (s_0, a_0, r_0, ..., s_T, a_T, r_T)
    Compute returns: G_t for each timestep
    for t = 0 to T:
        θ ← θ + α ∇_θ log π_θ(a_t|s_t) G_t
```

## 2. Characteristics of REINFORCE

### Advantages
1. **Simple**: Easy to understand and implement
2. **Theoretically sound**: Guaranteed convergence to local optima
3. **General**: Works with any differentiable policy
4. **No value function needed**: Can work without baseline

### Disadvantages
1. **High variance**: Uses full trajectory returns (high variance)
2. **Sample inefficient**: Only uses on-policy data
3. **Slow convergence**: Requires many episodes
4. **Reward scaling sensitive**: Performance depends on reward scale

### Complexity Analysis
- **Time per episode**: O(T) where T is episode length
- **Space**: O(T) for storing trajectory
- **Sample complexity**: O(1/ε²) for ε-optimal policy

## 3. Implementation Details

### Key Components

1. **Policy Network**: Maps states to action probabilities
   - Input: state
   - Output: action logits (for discrete) or mean/std (for continuous)

2. **Return Computation**: Compute discounted returns
   - $G_t = r_t + \gamma G_{t+1}$
   - Computed in reverse order for efficiency

3. **Policy Loss**: Negative log probability weighted by returns
   - $L = -\mathbb{E}[\log \pi_\theta(a|s) G_t]$

4. **Entropy Regularization**: Encourage exploration
   - $L_{total} = L_{policy} - \beta H(\pi)$
   - where $H(\pi) = -\mathbb{E}[\log \pi]$ is entropy

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical

# Simple policy network for CartPole
class SimplePolicy(nn.Module):
    def __init__(self, state_dim=4, action_dim=2, hidden_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, state):
        return self.net(state)
    
    def sample(self, state):
        """Sample action and return log probability"""
        if state.dim() == 1:
            state = state.unsqueeze(0)
        
        logits = self.forward(state)
        dist = Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        return action.squeeze(), log_prob.squeeze()

# Test the policy
policy = SimplePolicy()
state = torch.randn(4)
action, log_prob = policy.sample(state)
print(f"State shape: {state.shape}")
print(f"Action: {action.item()}")
print(f"Log probability: {log_prob.item():.4f}")

## 4. Return Computation

### Efficient Computation

Computing returns efficiently is crucial for training speed.

In [ ]:
def compute_returns(rewards, gamma=0.99):
    """
    Compute discounted returns from rewards.
    
    Args:
        rewards: List of rewards from trajectory
        gamma: Discount factor
    
    Returns:
        Array of returns for each timestep
    """
    returns = np.zeros(len(rewards))
    cumulative_return = 0.0
    
    # Compute returns in reverse order
    for t in reversed(range(len(rewards))):
        cumulative_return = rewards[t] + gamma * cumulative_return
        returns[t] = cumulative_return
    
    return returns

# Example
rewards = [1.0, 1.0, 1.0, 0.0]  # CartPole rewards
returns = compute_returns(rewards, gamma=0.99)

print("Rewards:", rewards)
print("Returns:", returns)
print("\nExplanation:")
print(f"  G_3 = 0")
print(f"  G_2 = 1 + 0.99 * 0 = 1.0")
print(f"  G_1 = 1 + 0.99 * 1.0 = 1.99")
print(f"  G_0 = 1 + 0.99 * 1.99 = 2.9701")

## 5. Training Loop

### Complete REINFORCE Training

In [ ]:
def train_reinforce_episode(policy, env, optimizer, gamma=0.99, entropy_coeff=0.01):
    """
    Train REINFORCE for one episode.
    
    Args:
        policy: Policy network
        env: Gym environment
        optimizer: PyTorch optimizer
        gamma: Discount factor
        entropy_coeff: Entropy regularization coefficient
    
    Returns:
        Episode return and loss
    """
    state, _ = env.reset()
    log_probs = []
    rewards = []
    entropies = []
    
    # Collect trajectory
    done = False
    while not done:
        state_tensor = torch.FloatTensor(state)
        
        # Sample action
        logits = policy(state_tensor.unsqueeze(0))
        dist = Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        entropy = dist.entropy()
        
        # Take environment step
        next_state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated
        
        log_probs.append(log_prob)
        rewards.append(reward)
        entropies.append(entropy)
        
        state = next_state
    
    # Compute returns
    returns = compute_returns(rewards, gamma)
    returns = torch.FloatTensor(returns)
    
    # Normalize returns
    returns = (returns - returns.mean()) / (returns.std() + 1e-8)
    
    # Compute loss
    log_probs = torch.stack(log_probs)
    entropies = torch.stack(entropies)
    
    policy_loss = -(log_probs * returns).mean()
    entropy_loss = -entropy_coeff * entropies.mean()
    total_loss = policy_loss + entropy_loss
    
    # Update policy
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    return sum(rewards), total_loss.item()

print("REINFORCE training function defined successfully!")

## 6. Summary

### Key Takeaways
1. **REINFORCE** is the foundational policy gradient algorithm
2. **Simple but high variance** - uses full trajectory returns
3. **On-policy** - requires collecting new data after each update
4. **Entropy regularization** helps with exploration

### Next Steps
- Add value function baseline to reduce variance
- Explore Actor-Critic methods
- Implement advanced variants (PPO, TRPO)